# Project Summary: Experiments on Image Orientation Estimation using Convolutional Neural Networks

#### Author: Riccardo Albertazzi
#### Project Work of Computer Vision and Image Processing M, University of Bologna, Italy -  Professor Luigi Di Stefano

The task of this project work is to explore the CNN scenario in order to understand how these networks behave when they have to classify images that are arbitrarily rotated. At the same time, these networks are required to classify images; therefore it's important to understand how the classification task is influenced by the amount of image rotations.

Here I'll explain briefly the different type of networks that I tested for estimating the orientation:
* Using a 2-values output to map the sine and cosine of the angle
* Using a classification output where each class corresponds to 1 angle degree
* Exploring how a network is able to predict unseen orientations using the 2-values output
* Using an autoencoder to estimate the class and the orientation

# 0. Configuration

Hardware: virtual machine on Google Cloud
* Intel CPU with 4 vCPU
* 15 GB RAM
* 1/2 Nvidia Tesla K80 (12 GB)

OS: Ubuntu 16.04 LTS

Software:
* CUDA 9.0 with CuDNN
* Python 3.5.2
* TensorFlow 1.5
* Keras 2.1.4
* OpenCV 3

# 1. Dataset: Fashion-MNIST

I performed some experiments with the well-known MNIST dataset. Since this dataset is very easy to solve (without orientation), and since not all digits are well aligned, I decided to switch to the Fashion-MNIST dataset from Zalando. This dataset contains 60000 grayscale images (50000 for training, 10000 for testing) of clothes (t-shirts, shoes, bags, purses, ...) divided in 10 classes. The pros of using this dataset for this project are the following:
* Images are 28x28, so I won't need to train very big and complex networks
* Images seem to be perfectly aligned horizontally, which is ideal for working with rotations

![](images/fashion-mnist.png)

# 2. Estimating image orientation by predicting sine and cosine of the angle

In my first attempt I trained a network that predicts the class of the image and its orientation by estimating the sine and cosine of the angle. The predicted angle can then be reconstructed by using the formula:

$$ angle_{degrees} = arctan(sine/cosine) \times 180 / \pi $$

This how the experiment works:
1. Define a rotation range $r$
2. For each image in the train/test set, create 3 randomly rotated versions of the image inside the range $[-r, r)$. The new train/test sets will be 3 times as big as the original sets and will contain the rotated versions of the original images. 
    * It is worth pointing out that images that originally belong to the training (test) set will end up in the rotated training (test) set; in other words, there is no overlap between the newly created training and test set. But both the training and the test set will contain all rotations inside the circular sector $[-r, r)$.
    * Images are zero padded (2 rows/columns each side) before rotating them. By doing that, we avoid losing part of the images during the rotation process. The newly created sets will therefore contain images that are 32x32 pixels. The optimal size that guarantees that no pixel will be lost is $28\sqrt{2} = 40 $, but I empirically chose a smaller size because it seemed enough to avoid losing meaningful parts of the image.
3. Compute sine and cosine of all generated rotations
4. Train a network with two outputs: the classification output and the sine-cosine output
5. Collect statistics

The process described above is repeated for increasing values of $r$, starting from $r = 0$ (no rotations) to $r = 180$ (full circle). $r$ is increased by 10 degrees every time.

This is the network that I trained:

![](images/model-rotation-continuous.png)

Additional considerations:
* The classification loss is the canonical categorical crossentropy, while the orientation output minimizes the dot product between the true and the predicted unit vector. In Keras this loss comes out of the box and is called *cosine proximity*:

In [ ]:
def cosine_proximity(y_true, y_pred):
    y_true = K.l2_normalize(y_true, axis=-1)
    y_pred = K.l2_normalize(y_pred, axis=-1)
    return -K.sum(y_true * y_pred, axis=-1)

* Sometimes the network wasn't properly training, meaning that the classification accuracy didn't go above 10%. To avoid that I had to use weighted losses for the first epoch. By doing that, we can first let the network concentrate on the classification task, and then work on the orientation:
    * During the first epoch, 90% of the total loss is due to the classification loss, while 10% is due to the orientation loss.
    * After the first epoch, the balance between the two losses is restored.

### 2.1 Results

The following graph shows how the classification accuracy and the rotation loss (that we can consider to be $-mean(cosine)$ between the predicted and the true angle) changes with the size of $r$ (actually, $2r$ has been graphed). The graph shows that both the classification and the orientation estimation become harder and harder task with the increasing range of rotations:

![](images/02_accuracy_loss.png)

The following graphs refer to the case $r = 180$ (full circle). The first graph shows the histogram of the absolute differences between predicted and true orientation, while the second shows the percentiles of these differences:

![](images/02_histogram.png)

![](images/02_percentile.png)

For the sake of completeness, I also report the confusion matrix of the 10 labels (case $r = 180$). We can see that nearly half of the classes are perfectly recognized while the other half are more problematic: 

![](images/02_confusion_matrix.png)

### 2.2 Variations on the orientation estimation

I tested three variations on the basic structure described above:
1. Use an activation function on the orientation output that clips the output values in the range $[-1, 1]$ (like *tanh*)
2. Add a penalization factor to the dot product loss that lets that predicted 2-dimensional vector to have unitary norm:

    $$ new\_loss = cosine\_proximity\_loss + (1 - (x_{pred}^2 + y_{pred}^2))^2 $$

3. Use an activation function that explicitly outputs a 2d vector with unitary norm. This is achieved using a Lambda layer in keras:

In [ ]:
predicted_orientation = layers.Lambda(lambda l: l / K.sqrt(K.sum(l**2)), name='r')(r)

The first two variations didn't allow for a good learning. The third variation was giving results comparable to the basic case, but the output values didn't have unitary norm (they were much smaller)

# 3. Estimating image orientation by using a classification output

The only difference between this experiment and the previous one lies in how the orientation is predicted. While in the previous case I was working with 2 continuous output that represented the sine and the cosine and the angle, here I worked on a classification task where each output neuron will tell the probability that the input image has a particular degree.

Since I worked with increasing values of rotation ranges (the $r$ parameter), and since each rotation output refers to 1 degree of rotation, the number of outputs grows with $r$: $output\_neurons = 2r$.

Again, this is the network that I trained:

![](images/model-rotation-categorical.png)

In this case, the rotation loss becomes a categorical crossentropy, just like the classification output.

### 3.1 Results

Again, both the classification and the orientation estimation decreases with increasing rotation ranges. However, the results seem to be more promising than the previous case:

![](images/03_accuracy_loss.png)

In this graph I show the mean angle difference in function of the rotation range for both the sin-cos output and the rotation-as-classification output. We can see that the latter behaves always better:

![](images/03_vs_02.png)

We can definitely see that if we take a look at the histogram of absolute differences and its percentiles (case $r = 180$):

![](images/03_histogram.png)

![](images/03_percentile.png)

# 4. Dealing with wrong predictions of multiples of 90 degrees

If we take a closer look at the last shown histogram, we can notice 2 peaks in proximity of the values 90 and 180. If we show the histogram in logarithmic scale, we obtain the following (this is a different training so the histogram will not be exactly the same):

![](images/03_histogram_log.png)

It seems that most of the wrong orientation detections lie in multiples of 90 degrees.

If we plot the images that whose orientation is estimated with an absolute error of, respectively, 90 and 180 degrees, we can see that many images have a squared or rectangular shape, and that may cause the network to fail to recognize symmetries. However, not all of them have strong symmetries.

Images whose orientation is estimated with an error of exactly 90 degrees:
![](images/03_wrong_90.png)

Images whose orientation is estimated with an error > 175 degrees:
![](images/03_wrong_180.png)

To understand if that is just a problem related to a network that has too many rotations to predict and understand, I synthesized samples using rotations that are only multiple of 30 degrees (0, 30, 60, ..., 330) and I built a network that predicts the orientation as a classification problem (1 output for each discrete rotation). This network obviously behave better than a network that has to recognize 360 degrees rotations, but the mistaken orientations that are multiple of 90 degrees are still present:

![](images/04_histogram.png)

If we plot the confusion matrix of the orientation estimation (not the class of the image!) we can clearly see how this problem represent the majority of the wrong guesses. We can in fact see lines that are parallel to the principal diagonal and represent wrong guesses that are multiple of 90 degrees:

![](images/04_confusion_matrix.png)

My other guess was that the network was learning small artifacts in the images that are rotated by a value that is not multiple of 90, but the previous plot shows that this problem is related to all possible rotations.

As another experiment, I tried to synthesize samples at multiples of a certain angle (just like in the previous example), but instead of using a single descrete value I used a small range of rotations that lie near that angle. The task of the network is therefore classifying orientations that belong to the same circular sector into the same class. This should have the following advantages:
* Let the network concentrate on the required rotations (just like the discrete case)
* Help the network classify images by using nearby orientations

Unfortunately, I wasn't able to solve the problem. If we take a look at this other confusion matrix (involving again image orientations), where the sectors have been placed at multiples of 10 degrees, we can perfectly see how the wrong classification involves images whose difference between the true and the predicted orientation is a multiple of 90 degrees.

![](images/05_confusion_matrix_10.png)

The problem remains unsolved.

# 5. Let a network predict unseen orientations

We have dealt so far with training and test sets that contain different images, but they both contain all possible rotations inside the target rotation range. Now we want to understand if a network not only is able to estimate the rotation of unseen images, but also if it's able to estimate a rotation that it has never seen during the training phase.

This is how the experiment works:
1. Define the size of a circular sector $s$ such that 360 is a multiple of $s$ and 360 *mod* $s$ is even.
2. Split the circle in adjacent circular sectors of size $s$. Training images will be rotated by values that lie into the even circular sectors, while testing images will be rotated by values that lie into the odd ones.
    * If the experiment works, the network should be able to estimate angles that lie inside the unseen circular sectors
    
3. Train a network on estimating the class and the orientation. Since we are dealing with unseen rotations we cannot use the categorical output (the network would probably learn to set to 0 the outputs that are always 0 in the training set), but we can still use the sin-cos output.
4. Collect data
5. Repeat for different values of $s$.

### 5.1 Results

As expected, the network doesn't behave well for very big circular sectors (> 20 degrees). When dealing with smaller circular sectors (<= 10 degrees), the network is still able to make good predictions on the orientation of the unseen rotated images.

![](images/06_accuracy_loss.png)

![](images/06_percentile.png)

# 6. Using an autoencoder to estimate image class and orientation

As a last experiment, I tried to train an autoencoder on the train images and then estimate the image class and orientation of the test samples by finding the nearest neighbor in the encoded space. This is the topology of the network:

![](autoencoder.png)

The layer that outputs the encoded vector of the image is *dense_1*. We are then trying to map a 32x32d vector into a 128-dimensional vector.

After the training, this is the output of the autoencoder for some test images:

![](images/autoencoder_images.png)

After training, we encode all training images into a matrix of size (N_TRAIN_SAMPLES, 128). In order to compute the class and the orientation of a test sample, I encode the test image and then I find the nearest neighbor of the test vector inside the train matrix. This is solved by using common machine learning algorithms; note that this approach is significantly slower than using a neural network on the GPU: on my virtual machine *KD-Tree* would take 3 hours to find the nearest neighbour for 30k testing samples. Fortunately, another algorithm named *Ball Tree* is available on the *scikit-learn* library that works much faster with higher dimensions (6 minutes for my 30k samples).

The results are very promising: the mean orientation error is 9 degrees, lower than the sin-cos approach, but higher than the categorical approach. We can see from the histogram that the problem of wrong orientation that are multiple of 90 or 180 degrees is still present:

![](images/07_histogram.png)

If we take a look at the graph of the percentiles, we can see that the curve is much closer to the categorical approach than to the sin-cos approach, meaning that the autoencoder is working very well.

![](images/07_percentile.png)

This type of network could be easily extended for other types of features, such as scale or intensity.

However the autoencoder approach doesn't work so well for the classification task, which gives a result of 77% (against 85% of the categorical approach).

# Summary

In this project I was able to test different neural network configurations for estimating the orientation and the class of rotated Fashion-MNIST images. These are my conclusions:
* Using a categorical output for estimating the orientation works better than performing a regression on the sine and cosine of the angle to be estimated
* The majority of the wrong angle estimations are due to differences between the true and the predicted angle that are multiple of 90 degrees. This could be caused by symmetries in some of the images that the network is not able to distinguish.
* When using the sin-cos output, the network is also able to estimate unseen angles, if they do not fall too far from the training angles.
* Using an autoencoder and solving a nearest neighbor problem in the latent space seems to be a good alternative for estimating the orientation of an image. 